# Feature Engineering
In der Datenanalyse ist aufgefallen das bei vielen Label nur ein einzelnes Wort zugeordnet ist. Um das zu verbessern wird in diesem Abschnitt für jedes Wort in Wikipedia nach einem passenden Beitrag gesucht um dann die ersten Sätze daraus zu verwenden als Feature für das Modell.

In [ ]:
#import librarys
import re
import tqdm
from datetime import datetime

import pandas as pd
import numpy as np
import seaborn as sns

import tensorflow as tf

from sklearn import metrics
from sklearn.model_selection import train_test_split


"\ntqdm.tqdm.pandas()\nsns.set_style('whitegrid')\n\npd.options.display.max_colwidth = 600\npd.options.display.max_rows = 400\n"

In [ ]:
df = pd.read_csv('drive/MyDrive/Colab Notebooks NLP/data/Total_AlyMan.csv')
df.shape

(5841, 2)

In [ ]:
df['Beispiel']

0           Erkennungsobjekt für elektrisches Potential
1       Objekt zur Erkennung von elektrischem Potential
2                                  Kopplungskondensator
3                                   Messspannungsrelais
4                       Spannungswandler für Messzwecke
                             ...                       
5836                           Luftkanalqualitätsfühler
5837                                             Dimmer
5838                              Kabeltemperaturfühler
5839                      Luftkanaldruckdifferenzfühler
5840                          Luftkanaltemperaturfühler
Name: Beispiel, Length: 5841, dtype: object

In [ ]:
# enshure to delete leading and trailing whitespaces
df['Beispiel'] = df['Beispiel'].str.strip()

# German Compound Splitter
In einem ersten Schritt müssen die zusammengesetzten Wörter wie "Luftkanalqualtitätsfühler" getrennt werden da es sonst keine Einträge dazu in Wikipedia gibt. Dazu wird das package german_compound_spliter verwendet.

https://github.com/repodiac/german_compound_splitter

In [ ]:
!pip install git+https://github.com/repodiac/german_compound_splitter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/repodiac/german_compound_splitter to /tmp/pip-req-build-l7plzrew
  Running command git clone -q https://github.com/repodiac/german_compound_splitter /tmp/pip-req-build-l7plzrew
     |████████████████████████████████| 110 kB 5.2 MB/s 
  Created wheel for german-compound-splitter: filename=german_compound_splitter-0.1.0-py3-none-any.whl size=14022 sha256=30de385de1e1189d172d035d4660a499f0f8c81af0d493e21246e3cad7448746
  Stored in directory: /tmp/pip-ephem-wheel-cache-_rpqt7bp/wheels/5a/2f/3c/2e1ce528466ed5c4704996b98e675f728da3cf5b7438bbaea6
Successfully built german-compound-splitter


In [ ]:
from german_compound_splitter import comp_split

# Load german dictionary
input_file = "drive/MyDrive/Colab Notebooks NLP/data/compound_spliter/german.dic"
ahocs = comp_split.read_dictionary_from_file(input_file)

Loading data file - drive/MyDrive/Colab Notebooks NLP/data/compound_spliter/german.dic


In [ ]:
# Test sample, split dissect split words in theyr base form
compound = 'Donaudampfschifffahrtskapitänsmützenabzeichen'
dissection = comp_split.dissect(compound, ahocs, make_singular=True)
print('SPLIT WORDS (plain):', dissection)
print('SPLIT WORDS (post-merge):', comp_split.merge_fractions(dissection))

Dissect compound:  Donaudampfschifffahrtskapitänsmützenabzeichen
SPLIT WORDS (plain): ['Donau', 'Dampf', 'Schifffahrt', 'Kapitän', 'Mütze', 'Abzeichen']
SPLIT WORDS (post-merge): ['Donau', 'Dampf', 'Schifffahrt', 'Kapitän', 'Mütze', 'Abzeichen']


In [ ]:
df["wordsplit"] = np.nan

In [ ]:
%%capture output
text = ''
error = []
indx = 0

for row in range(len(df)):
  text = df["Beispiel"][row]
  # only entrys with one word will be splittet
  if len(text.split()) == 1:
    try:
      df["wordsplit"][indx] = comp_split.dissect(text, ahocs, make_singular=True)
    except:
      error.append(text)
  indx  += 1

In [ ]:
df.head(10)

,Kennzeichen,Beispiel,wordsplit
0,BA,Erkennungsobjekt für elektrisches Potential,NaN
1,BA,Objekt zur Erkennung von elektrischem Potential,NaN
2,BA,Kopplungskondensator,"[Kopplung, Kondensator]"
3,BA,Messspannungsrelais,"[Messspannung, Relais]"
4,BA,Spannungswandler für Messzwecke,NaN
5,BA,Spannungsrelais,"[Spannung, Relais]"
6,BA,Spannungswandler,"[Spannung, Wandler]"
7,BB,Erkennungsobjekt für Widerstand oder Leitfähigkeit,NaN
8,BB,Objekt zur Erkennung von Widerstand,NaN
9,BB,Detektor für elektrischen Widerstand,NaN


In [ ]:
# With the function merge_fractions we check if new words can be created 
df["merge"] = df.apply(lambda row: comp_split.merge_fractions(row["wordsplit"]) if type(row["wordsplit"])==list else None, axis=1)

In [ ]:
df.head(10)

,Kennzeichen,Beispiel,wordsplit,merge
0,BA,Erkennungsobjekt für elektrisches Potential,NaN,None
1,BA,Objekt zur Erkennung von elektrischem Potential,NaN,None
2,BA,Kopplungskondensator,"[Kopplung, Kondensator]","[Kopplung, Kondensator]"
3,BA,Messspannungsrelais,"[Messspannung, Relais]","[Messspannung, Relais]"
4,BA,Spannungswandler für Messzwecke,NaN,None
5,BA,Spannungsrelais,"[Spannung, Relais]","[Spannung, Relais]"
6,BA,Spannungswandler,"[Spannung, Wandler]","[Spannung, Wandler]"
7,BB,Erkennungsobjekt für Widerstand oder Leitfähigkeit,NaN,None
8,BB,Objekt zur Erkennung von Widerstand,NaN,None
9,BB,Detektor für elektrischen Widerstand,NaN,None


In [ ]:
#  Now we add all the new created words in one column
%%capture output
ind = 0
df["joind"] = None
for row in range(len(df)):
  df['joind'][ind] = []
  if type(df["merge"][ind])==list:
    for x in range(len(df["merge"][ind])):
      df['joind'][ind].append(df["merge"][ind][x])
    df['joind'][ind].append(df["Beispiel"][ind])
  else:
    df['joind'][ind].append(df["Beispiel"][ind])
  ind += 1

In [ ]:
df.head(10)

,Kennzeichen,Beispiel,wordsplit,merge,joind
0,BA,Erkennungsobjekt für elektrisches Potential,NaN,None,[Erkennungsobjekt für elektrisches Potential]
1,BA,Objekt zur Erkennung von elektrischem Potential,NaN,None,[Objekt zur Erkennung von elektrischem Potential]
2,BA,Kopplungskondensator,"[Kopplung, Kondensator]","[Kopplung, Kondensator]","[Kopplung, Kondensator, Kopplungskondensator]"
3,BA,Messspannungsrelais,"[Messspannung, Relais]","[Messspannung, Relais]","[Messspannung, Relais, Messspannungsrelais]"
4,BA,Spannungswandler für Messzwecke,NaN,None,[Spannungswandler für Messzwecke]
5,BA,Spannungsrelais,"[Spannung, Relais]","[Spannung, Relais]","[Spannung, Relais, Spannungsrelais]"
6,BA,Spannungswandler,"[Spannung, Wandler]","[Spannung, Wandler]","[Spannung, Wandler, Spannungswandler]"
7,BB,Erkennungsobjekt für Widerstand oder Leitfähigkeit,NaN,None,[Erkennungsobjekt für Widerstand oder Leitfähigkeit]
8,BB,Objekt zur Erkennung von Widerstand,NaN,None,[Objekt zur Erkennung von Widerstand]
9,BB,Detektor für elektrischen Widerstand,NaN,None,[Detektor für elektrischen Widerstand]


# Wikipedia API
Nun, da wir eine Spalte mit Wörtern haben, können wir die Wikipedia-Api verwenden, um einige Inhalte für diese Wörter zu erhalten.

https://www.jcchouinard.com/wikipedia-api/#Wikipedia_Actions

In [ ]:
# libary imports
import requests
import re

In [ ]:
pattern1 = r'\(.*?\)'
pattern2 = r'\[.*?\]'
url = 'https://de.wikipedia.org/w/api.php'

In [ ]:
tx =  " asdfjaf [arqwed.asdgqwe] asdfqwe"

params = {
        'action': 'query',
        'format': 'json',
        'prop': 'extracts',
        'exintro': True,
        'explaintext': True,
    }

In [ ]:
subject = 'Sensor'
params = {
        'action': 'query',
        'format': 'json',
        'titles': subject,
        'prop': 'extracts',
        'exintro': True,
        'explaintext': True,
    }
response = requests.get(url, params=params)
data = response.json()

In [ ]:
#function to create a api request to wikipedia
def getWikiAbstract(subject: str , maxWords = 25, maxSentence = 1):
  params = {
        'action': 'query',
        'format': 'json',
        'titles': subject,
        'prop': 'extracts',
        'exintro': True,
        'explaintext': True,
    }
  response = requests.get(url, params=params)
  data = response.json()
  abstract = next(iter(data['query']['pages'].values()))['extract']
  abstract = re.sub(pattern1, '', abstract)
  abstract = re.sub(pattern2, '', abstract)
  abstract = " ".join(abstract.split('.')[:maxSentence])
  return " ".join(abstract.split()[:maxWords])


In [ ]:
df["joind"][:20]

0            [Erkennungsobjekt für elektrisches Potential]
1        [Objekt zur Erkennung von elektrischem Potential]
2            [Kopplung, Kondensator, Kopplungskondensator]
3              [Messspannung, Relais, Messspannungsrelais]
4                        [Spannungswandler für Messzwecke]
5                      [Spannung, Relais, Spannungsrelais]
6                    [Spannung, Wandler, Spannungswandler]
7     [Erkennungsobjekt für Widerstand oder Leitfähigkeit]
8                    [Objekt zur Erkennung von Widerstand]
9                   [Detektor für elektrischen Widerstand]
10                    [Sensor für elektrischen Widerstand]
11                   [Detektor für thermischen Widerstand]
12               [Erkennungsobjekt für elektrischen Strom]
13           [Objekt zur Erkennung von elektrischem Strom]
14                         [elektronisches Überlastrelais]
15                    [Messstrom, Relais, Messstromrelais]
16                            [Stromwandler fürMesszweck

In [ ]:
# Create a new column and check for every row in "abstract if there is an entry
# in wikipedia. if it exist take the first sentence or the first 25 words
# This takes some time!
df["abstract"] = ""
for row in range(len(df)):
    try:
      df["abstract"][row] = getWikiAbstract(df["joind"][i][-1]).replace("\n", "")
      #print("1Gut - - ", df["joind"][i][-1]," ---", getWikiAbstract(df["joind"][i][-1]).replace("\n", ""))
    except:
      for word in range(len(df["joind"][row])-1):
        #abstarct = ""
        try:
          df["abstract"][row] = df["abstract"][row] + getWikiAbstract(df["joind"][row][word]).replace("\n", "") + '. '
          #abstract = abstract +  getWikiAbstract(df["joind"][row][word]).replace("\n", "") + '. '
          #print("2Gut - - ", df["joind"][i][ii]," ---", abstract)
        except:
          None
          #print("Error - - ", df["joind"][i][ii])

In [ ]:
#######################################
# Export to CSV
#######################################
from pathlib import Path
file_name = "drive/MyDrive/Colab Notebooks NLP/data/Total_FEN.csv"
file_path = (Path().resolve() / file_name).as_posix()
df.to_csv(file_path, index=False)